In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
import copy
import json
import pandas as pd

In [22]:
urls=pd.read_csv("中医古书_普济方.csv")  

In [39]:
def contain_chinese(check_str):
    """
    判断字符串中是否包含中文
    :param check_str: {str} 需要检测的字符串
    :return: {bool} 包含返回True， 不包含返回False
    """
    for ch in check_str:
        if u'\u4e00' <= ch <= u'\u9fff':
            return True
    return False

In [45]:
def get(url):
    r=requests.get(url,headers={
        'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.100 Safari/537.36',
        # 这里还可以以字典形式给出别的请求头属性
        })
    r.encoding='gbk'
    soup=BeautifulSoup(r.text,'lxml')
    t=soup.select('body')
    
    a=str(t[0])
    
    a=a.replace('\n','')
    a=a.replace('\r','')
    a=a.replace('<br/>','(br/)')
    
    a=a.split('</')
    
    for i in range(0,len(a)):
        a[i]=a[i].replace('<p>','')
    
    for i in range(0,len(a)):
        a[i]=a[i].replace('p>','')
    
    for i in range(0,len(a)):
        a[i]=a[i].replace('<span>','(span)')
    
    for i in range(0,len(a)):
        a[i]=a[i].replace('span>','')
    
    for i in range(0,len(a)):
        a[i]=a[i].replace('<div>','')
    
    for i in range(0,len(a)):
        a[i]=a[i].replace('<h2>','h2>')
    
    for i in range(0,len(a)):
        a[i]=a[i].replace('<hr width="60%"/>','')
    
    for i in range(0,len(a)):
        a[i]=a[i].replace('h4>(span)','(span)')
    
    b=[]
    for i in range(0,len(a)):
        if a[i].count('(span)')>1:
            temp=a[i].split('(span)')
            for j in temp:
                if j!='':
                    b.append('(span)'+j)
                
        else:
            b.append(a[i])
    
    c=[]
    for i in range(0,len(b)):
        c.extend(b[i].split('<'))
        
    
    #pattern = '[\u4e00-\u9fa5]{0,}'    
    data=[]
    for i in c:
        if i!='' and i!='\n':
            i=i.replace('\n','')
            #mat=re.match(pattern,i)
            #if len(mat.group())!=0:
            if contain_chinese(i):
                data.append(i)
            
    return(data)

In [46]:
dic={}
for i in range(8,len(urls)):
    #time.sleep(60)
    url=urls['字段1_文本'][i]
    name=urls['名称'][i]
    data=get(url)
    dic[name]=data
    print(name)

- 14 肝脏门
- 16 心脏门
- 20 脾脏门
- 26 肺脏门
- 29 肾脏门
- 34 胆腑门
- 35 胃腑门
- 37 大肠腑门
- 41 小肠腑门
- 42 膀胱腑门
- 43 三焦腑门
- 44 头门
- 51 面门
- 53 耳门
- 56 鼻门
- 58 口门
- 59 舌门
- 60 咽喉门
- 65 牙齿门
- 71 咽喉门
- 87 诸风门
- 106 诸风门
- 117 寒暑湿门
- 119 积热痼冷门
- 121 寒暑湿门
- 132 伤寒门
- 148 时气门
- 152 热病门
- 154 身体门
- 157 咳嗽门
- 164 痰饮门
- 168 积聚门
- 176 消渴门
- 181 诸气门
- 185 诸痹门
- 188 诸血门
- 191 水病门
- 195 黄疸门
- 197 诸疟门
- 201 霍乱门
- 204 膈噎门
- 206 呕吐门
- 207 泄痢门
- 214 小便淋秘门
- 217 诸虚门
- 227 虚劳门
- 235 劳瘵门
- 237 尸疰门
- 239 诸虫门
- 240 脚气门
- 247 疝门
- 250 诸毒门
- 254 杂治门
- 257 食治门
- 260 乳石门
- 263 服饵门
- 267 诸汤香煎门
- 268 杂录门
- 272 诸疮肿门
- 282 痈疽门
- 291 痈疽门
- 294 瘿瘤门
- 295 痔漏门
- 299 上部疮门
- 301 下部疮门
- 302 金疮门
- 304 刺疮门
- 305 杖疮门
- 306 诸虫兽伤门
- 309 折伤门
- 313 膏药门
- 316 妇人诸疾门
- 336 妊娠诸疾门
- 345 产后诸疾门
- 356 产难门


In [51]:
#爬取的数据存入json
filename='new测试4.15.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(dic,file_obj,ensure_ascii=False,indent = 4)

In [52]:
#读取json中爬取的数据
with open('new测试4.15.json',encoding="utf-8") as file_obj:
    dic=json.load(file_obj)

In [54]:
temp_dict={}
for key in dic.keys():
    temp_dict[key]={}
    last_name=''
    last_index=0
    flag=0
    for i in range(0,len(dic[key])):
        if 'h1'==dic[key][i][:2]:
            #print(dic[key][i])
            if flag==0:
                temp_dict[key]['总']=dic[key][:i]
                last_name=dic[key][i]
                #print(last_name)
                last_index=i
                #print(last_index)
                flag+=1
            else:
                #print('else',last_name)
                #print('else',last_index)
                temp=dic[key][last_index+1:i]
                temp_dict[key][last_name]=temp
                last_name=dic[key][i]
                last_index=i
                flag+=1
    temp_dict[key][last_name]=dic[key][last_index+1:]

In [57]:
#爬取的数据存入json
filename='new测试4.16.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(temp_dict,file_obj,ensure_ascii=False,indent = 4)

In [6]:
#读取json中爬取的数据
with open('new测试4.16.json',encoding="utf-8") as file_obj:
    temp_dict=json.load(file_obj)

In [7]:
for key in temp_dict.keys():
    for name in list(temp_dict[key].keys()):
        if 'h1' in name:
            st=name.replace('h1>','')
            temp_dict[key][st]=temp_dict[key][name]
            del temp_dict[key][name]

In [3]:
#爬取的数据存入json
filename='new测试4.17.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(temp_dict,file_obj,ensure_ascii=False,indent = 4)

NameError: name 'temp_dict' is not defined

In [13]:
#读取json中爬取的数据
with open('new测试4.17_1.json',encoding="utf-8") as file_obj:
    temp_dict=json.load(file_obj)

In [14]:
for key in temp_dict.keys():
    for name in temp_dict[key].keys():
        temp=temp_dict[key][name]
        c=[]
        flag=0
        last_index=0
        for i in range(0,len(temp)):
            if temp[i][:2]=='h2':
                c.append(temp[last_index:i])
                last_index=i
        c.append(temp[last_index:])
        temp_dict[key][name]=c

In [20]:
#爬取的数据存入json
filename='new测试4.17_2.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(temp_dict,file_obj,ensure_ascii=False,indent = 4)

In [32]:
#读取json中爬取的数据
with open('new测试4.17_2.json',encoding="utf-8") as file_obj:
    temp_dict=json.load(file_obj)

In [34]:
for key in temp_dict.keys():
    for name in temp_dict[key].keys():
        temp=temp_dict[key][name]
        temp_d={}
        for i in temp:
            if len(i)==1:
                if '总' in temp_d:
                    temp_d['总']['论'].extend(i[0])
                else:
                    temp_d['总']={}
                    temp_d['总']['论']=i[0]
            else:
                if 'h2>' in i[0]:
                    if i[0] in temp_d:
                        temp_d[i[0]]['内容'].extend(i[1:])
                    else:
                        temp_d[i[0]]={}
                        temp_d[i[0]]['内容']=i[1:]
                else:
                    if '附论' in temp_d:
                        temp_d['附论']['内容'].extend(i[1:])
                    else:
                        temp_d['附论']={}
                        temp_d['附论']['内容']=i[1:]
        temp_dict[key][name]=temp_d

In [35]:
#爬取的数据存入json
filename='new测试4.17_3.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(temp_dict,file_obj,ensure_ascii=False,indent = 4)

In [ ]:
无词典的分词
QA系统（限定领域的问答系统
给一本书，对书的内容进行理解，然后对书内容进行提问

In [98]:
#读取json中爬取的数据
with open('new测试4.17_3.json',encoding="utf-8") as file_obj:
    temp_dict=json.load(file_obj)

In [99]:
new_dict={}
for key in temp_dict.keys():
    new_dict[key]={}
    for name in temp_dict[key].keys():
        new_dict[key][name]=[]
        temp_lst=[]
        for cate in temp_dict[key][name].keys():
            temp_lst.append(cate)
            if '内容' in temp_dict[key][name][cate]:
                temp=temp_dict[key][name][cate]['内容']
            elif '论' in temp_dict[key][name][cate]:
                temp=temp_dict[key][name][cate]['论']
            if type(temp)==list:
                for i in temp:
                    temp_lst.append(i)
            else:
                temp_lst.append(temp)
        new_dict[key][name]=temp_lst

In [100]:
#爬取的数据存入json
filename='new测试4.19.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(new_dict,file_obj,ensure_ascii=False,indent = 4)

In [101]:
#读取json中爬取的数据
with open('new测试4.19.json',encoding="utf-8") as file_obj:
    new_dict=json.load(file_obj)

In [102]:
for key in new_dict.keys():
    for name in new_dict[key].keys():
        temp=new_dict[key][name]
        c=[]
        flag=0
        last_index=0
        for i in range(0,len(temp)):
            if temp[i][:2]=='h2':
                c.append(temp[last_index:i])
                last_index=i
        c.append(temp[last_index:])
        new_dict[key][name]=c

In [103]:
#爬取的数据存入json
filename='new测试4.19.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(new_dict,file_obj,ensure_ascii=False,indent = 4)

In [104]:
for key in new_dict.keys():
    for name in new_dict[key].keys():
        temp=new_dict[key][name]
        temp_d={}
        for i in temp:
            if len(i)==1:
                if '总' in temp_d:
                    temp_d['总']['论'].extend(i[0])
                else:
                    temp_d['总']={}
                    temp_d['总']['论']=i[0]
            elif len(i)>1:
                if 'h2>' in i[0]:
                    if i[0] in temp_d:
                        temp_d[i[0]]['内容'].extend(i[1:])
                    else:
                        temp_d[i[0]]={}
                        temp_d[i[0]]['内容']=i[1:]
                else:
                    if '附论' in temp_d:
                        temp_d['附论']['内容'].extend(i[1:])
                    else:
                        temp_d['附论']={}
                        temp_d['附论']['内容']=i[1:]
        new_dict[key][name]=temp_d

In [105]:
#爬取的数据存入json
filename='new测试4.19_2.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(new_dict,file_obj,ensure_ascii=False,indent = 4)

In [106]:
new_dict2={}
for key in new_dict.keys():
    new_dict2[key]={}
    for name in new_dict[key].keys():
        new_dict2[key][name]={}
        for cate in new_dict[key][name].keys():
            temp=new_dict[key][name][cate]
            if 'h2>' in cate:
                nname=cate.replace('h2>','')
                new_dict2[key][name][nname]=temp
            else:
                new_dict2[key][name][cate]=temp

In [107]:
#爬取的数据存入json
filename='new测试4.19_2.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(new_dict2,file_obj,ensure_ascii=False,indent = 4)

In [108]:
#总论和总 归一
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if cate=='总论':
                new_dict2[key][name][cate]['论']=new_dict2[key][name][cate]['内容']
                del new_dict2[key][name][cate]['内容']

In [109]:
#爬取的数据存入json
filename='new测试4.19_2.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(new_dict2,file_obj,ensure_ascii=False,indent = 4)

## 每个方切割出来

In [2]:
#读取json中爬取的数据
with open('new测试4.19_2.json',encoding="utf-8") as file_obj:
    new_dict2=json.load(file_obj)

In [3]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '内容' in new_dict2[key][name][cate]:
                temp=new_dict2[key][name][cate]['内容']
            else:
                continue
            new=[]
            #count=0
            last_index=0
            for i in range(0,len(temp)):
                if 'h4>方' in temp[i]:
                    new.append(temp[last_index:i])
                    last_index=i+1
            new.append(temp[last_index:])
            new_dict2[key][name][cate]=new

In [4]:
#爬取的数据存入json
filename='new测试4.19_3.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(new_dict2,file_obj,ensure_ascii=False,indent = 4)

In [12]:
#读取json中爬取的数据
with open('new测试4.19_3.json',encoding="utf-8") as file_obj:
    new_dict2=json.load(file_obj)

In [13]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            temp=new_dict2[key][name][cate]
            new={}
            if type(temp)==list:
                for i in range(0,len(temp)):
                    if len(temp[i])==1:
                        if '论' not in new:
                            new['论']=temp[i]
                        else:
                            new['论'].extend(temp[i])
                    elif len(temp[i])>1:
                        if '方' not in new:
                            new['方']=temp[i]
                        else:
                            new['方'].extend(temp[i])
            else:
                new=temp
            new_dict2[key][name][cate]=new

In [14]:
#爬取的数据存入json
filename='new测试4.20.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(new_dict2,file_obj,ensure_ascii=False,indent = 4)

In [15]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate]:
                temp=new_dict2[key][name][cate]['方']
                new_lst=[]
                last_index=0
                for i in range(0,len(temp)):
                    if '(span)' in temp[i]:
                        new_lst.append(temp[last_index:i])
                        last_index=i
                new_lst.append(temp[last_index:])
                new_dict2[key][name][cate]['方']=new_lst

In [16]:
#爬取的数据存入json
filename='new测试4.20_1.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(new_dict2,file_obj,ensure_ascii=False,indent = 4)

In [28]:
#读取json中爬取的数据
with open('new测试4.20_1.json',encoding="utf-8") as file_obj:
    new_dict2=json.load(file_obj)

In [29]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                if len(temp[0])>=1:
                    if 'span' not in temp[0][0]:
                        new_dict2[key][name][cate]['论']=temp[0]
                        del new_dict2[key][name][cate]['方'][0]

In [30]:
#爬取的数据存入json
filename='new测试4.20_2.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(new_dict2,file_obj,ensure_ascii=False,indent = 4)

In [12]:
#读取json中爬取的数据
with open('new测试4.20_2.json',encoding="utf-8") as file_obj:
    new_dict2=json.load(file_obj)

#### 有些span被拆开了 (span)附矾丸 检查那些只有一行的span

In [17]:
#(span)混在句子后面的情况
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if 'span' in temp[i][0] and '(span'!=temp[i][0][0:5]:
                        index=temp[i][0].index('(span)')
                        temp[i-1].append(temp[i][0][:index])
                        temp[i][0]=temp[i][0][index:]

In [21]:
#爬取的数据存入json
filename='new测试4.20_3.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(new_dict2,file_obj,ensure_ascii=False,indent = 4)

In [47]:
#读取json中爬取的数据
with open('new测试4.20_3.json',encoding="utf-8") as file_obj:
    new_dict2=json.load(file_obj)

In [ ]:
#切分 方名和（出自）

In [48]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    new=[]
                    if '（' in temp[i][0] and temp[i][0].count('（')==1 and ' ' not in temp[i][0]:
                        index1=temp[i][0].index('（')
                        index2=temp[i][0].index('）')
                        new.append(temp[i][0][:index1])
                        new.append(temp[i][0][index1:index2+1])
                        if temp[i][0][index2+1:]!='':
                            new.append(temp[i][0][index2+1:])
                        for j in temp[i][1:]:
                            new.append(j)
                    else:
                        for j in temp[i][:]:
                            new.append(j)
                            
                    temp[i]=new

In [49]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                new=[]
                j=0
                while j<len(temp):
                    i=temp[j]
                    if len(i)==2 and ' ' not in i[0] and '（一名'==i[1][:3] and '）'==i[1][-1] and i[1].count('）')==1:
                        lst=i+temp[j+1]
                        new.append(lst)
                        j+=2
                    else:
                        new.append(i)
                        j+=1
                new_dict2[key][name][cate]['方']=new

In [ ]:
#先把剩下有句号的处理掉 

In [50]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in temp:
                    if len(i)==2 and 'span' in i[0] and ' ' not in i[0] and '（出'==i[1][:2] and '）'==i[1][-1] and i[1].count('）')==1:
                        if '。' in i[0]:
                            print(i)

In [ ]:
#然后处理(出xxxx)的 把跟后面的内容合在一起

['(span)皂盐散', '（出圣济总录）']
['(span)升麻散', '（出圣济总录）']
['(span)谷精草散', '（出圣济总录）']
['(span)柳枝膏', '（出圣济总录）']
['(span)麝香散', '（出圣济总录）']
['(span)胡桐泪散', '（出圣济总录）']
['(span)脆坏欲尽方', '（出圣济总录）']
['(span)防风饮', '（出圣济总录）']
['(span)丁香膏', '（出圣济总录）']
['(span)莽草散', '（出御药院）']
['(span)羌活散', '（出海上方）']
['(span)升麻散', '（出圣济总录）']
['(span)升麻散', '（出经验良方）']
['(span)麝香白牙散', '（出德生堂）']
['(span)定愈散', '（出圣惠方）']
['(span)升麻散', '（出圣惠方）']
['(span)槐枝八仙散', '（出御药院）']
['(span)槟榔散', '（出御药院）']
['(span)白茯苓', '（出神效方）']
['(span)雄黄散', '（出医方大成）']
['(span)细辛散', '（出圣惠方一名含漱汤）']
['(span)草乌头散', '（出圣济总录）']
['(span)柳枝汤', '（出圣济总录）']
['(span)升麻散', '（出御药院）']
['(span)生地黄散', '（出御药院）']
['(span)归荆散', '（出永类钤方）']
['(span)温风散', '（出直指方）']
['(span)开笑散', '（出圣济总录）']
['(span)细辛散', '（出圣济总录）']
['(span)立胜散', '（出御药院）']
['(span)一字散', '（出御药院）']
['(span)乳香定痛散', '（出御药院）']
['(span)透关散', '（出杨氏家藏方）']
['(span)甘松香散', '（出范氏方）']
['(span)地骨皮散', '（出圣惠方）']
['(span)鸡舌香散', '（出十便良方）']
['(span)防风散', '（出圣济总录）']
['(span)雄黄定痛散', '（出医方大成）']
['(span)露蜂房散', '（出海上方）']
['(span)齐蜂川椒散', '（出

['(span)花乳石散', '（出圣济总录）']
['(span)五物汤', '（出圣济总录）']
['(span)绿云汤', '（出圣济总录）']
['(span)二黄散', '（出圣济总录）']
['(span)硫黄散', '（出圣惠方）']
['(span)神验散', '（出圣济总录）']
['(span)木瓜丸', '（出圣济总录）']
['(span)木香散', '（出圣济总录）']
['(span)四妙散', '（出圣济总录）']
['(span)四香汤', '（出圣济总录）']
['(span)大豆帛方', '（出千金方）']
['(span)熏法', '（出千金方）']
['(span)熨药方', '（出圣惠方）']
['(span)淋渫药威灵仙散', '（出御药院方）']
['(span)乌荆丸', '（出宣明论）']
['(span)酒蒸黄连丸', '（出德生堂）']
['(span)槐黄散', '（出圣惠方）']
['(span)敷痔方', '（出直指方）']
['(span)槿花散', '（出百一选方）']
['(span)乌金散', '（出经验良方）']
['(span)蜗牛散', '（出危氏方）']
['(span)小黄连丸', '（出圣济总录）']
['(span)地肤子散', '（出圣济总录）']
['(span)驼粪熏痔方', '（出圣济总录）']
['(span)淋渫方', '（出圣济总录）']
['(span)扁竹方', '（出肘后方）']
['(span)吊肠丸', '（出朱氏集验方）']
['(span)黑雪膏', '（出朱氏集验方）']
['(span)木鳖散', '（出卫生家宝方）']
['(span)落痔膏', '（出卫生家宝方）']
['(span)立效散', '（出卫生家宝方）']
['(span)紫金膏', '（出卫生家宝方）']
['(span)半夏散', '（出卫生家宝方）']
['(span)夏枯草散', '（出卫生家宝方）']
['(span)蝉蜕散', '（出卫生家宝方）']
['(span)矾硝散', '（出卫生家宝方）']
['(span)妙应散', '（出卫生家宝方）']
['(span)龙脑散', '（出杨氏家藏方）']
['(span)槟榔散', '（出鲍氏方）']
['(span)内补丸'

In [51]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                new=[]
                j=0
                while j<len(temp)-1:
                    i=temp[j]
                    if len(i)==2 and 'span' in i[0] and ' ' not in i[0] and '（出'==i[1][:2] and '）'==i[1][-1] and i[1].count('）')==1:
                        lst=i+temp[j+1]
                        new.append(lst)
                        j+=2
                    else:
                        new.append(i)
                        j+=1
                new_dict2[key][name][cate]['方']=new

In [52]:
#爬取的数据存入json
filename='new测试4.23.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(new_dict2,file_obj,ensure_ascii=False,indent = 4)

# 4.24

In [20]:
#读取json中爬取的数据
with open('new测试4.23.json',encoding="utf-8") as file_obj:
    new_dict2=json.load(file_obj)

#### 切割 方跟主治在一行的切开

In [21]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if len(temp[i])==1 and len(temp[i][0])<=50 and ' ' in temp[i][0]:
                        new=[]
                        index=temp[i][0].index(' ')
                        new.append(temp[i][0][:index])
                        new.append(temp[i][0][index+1:])
                        temp[i]=new

In [22]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if len(temp[i])==1 and '治' in temp[i][0] and ' 治' not in temp[i][0] and '(span)治' not in temp[i][0] and '散治' in temp[i][0]:
                        new=[]
                        index=temp[i][0].index('治')
                        new.append(temp[i][0][:index])
                        new.append(temp[i][0][index:])
                        temp[i]=new

In [23]:
#爬取的数据存入json
filename='临时.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(new_dict2,file_obj,ensure_ascii=False,indent = 4)

In [24]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if len(temp[i])==1 and '治' in temp[i][0] and ' 治' not in temp[i][0] and '丸治' in temp[i][0]:
                        new=[]
                        index=temp[i][0].index('治')
                        new.append(temp[i][0][:index])
                        new.append(temp[i][0][index:])
                        temp[i]=new

In [25]:
#爬取的数据存入json
filename='临时.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(new_dict2,file_obj,ensure_ascii=False,indent = 4)

In [26]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if len(temp[i])==1 and ('汤治' in temp[i][0] or '膏治' in temp[i][0] or '丹治' in temp[i][0] or '饮治' in temp[i][0] or '酒治' in temp[i][0] or '方治' in temp[i][0]):
                        new=[]
                        index=temp[i][0].index('治')
                        new.append(temp[i][0][:index])
                        new.append(temp[i][0][index:])
                        temp[i]=new

In [27]:
#爬取的数据存入json
filename='临时.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(new_dict2,file_obj,ensure_ascii=False,indent = 4)

# 4.25

In [ ]:
#疗和治开头带空格 切分

In [28]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if len(temp[i])==1 and (' 治' in temp[i][0] or ' 疗' in temp[i][0]):
                        new=[]
                        if ' 治' in temp[i][0]:
                            index=temp[i][0].index(' 治')
                        elif ' 疗' in temp[i][0]:
                            index=temp[i][0].index(' 疗')
                        new.append(temp[i][0][:index])
                        new.append(temp[i][0][index+1:])
                        temp[i]=new

In [30]:
#疗开头的治疗切分
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if len(temp[i])==1 and '疗' in temp[i][0] and ('膏疗' in temp[i][0] or '丸疗' in temp[i][0] or '汤疗' in temp[i][0]):
                        new=[]
                        index=temp[i][0].index('疗')
                        new.append(temp[i][0][:index])
                        new.append(temp[i][0][index:])
                        temp[i]=new

In [32]:
#爬取的数据存入json
filename='new测试4.25.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(new_dict2,file_obj,ensure_ascii=False,indent = 4)

In [35]:
#读取json中爬取的数据
with open('new测试4.25.json',encoding="utf-8") as file_obj:
    new_dict2=json.load(file_obj)

## 4.26

In [ ]:
#把几个列表分开的连在一起

In [36]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if len(temp[i])==1 and '(span)治' not in temp[i][0] and '(span)疗' not in temp[i][0] and '。' in temp[i][0] and ('汤 ' not in temp[i][0] and '膏 ' not in temp[i][0] and '丸 ' not in temp[i][0] and '散 ' not in temp[i][0] and '饮 ' not in temp[i][0] and '丹 ' not in temp[i][0] ):
                        if len(temp[i-1])==1:
                            print(temp[i])

## 4.26 2

In [37]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if len(temp[i])==1 and '(span)治' not in temp[i][0] and '(span)疗' not in temp[i][0] and '。' in temp[i][0] and ('汤 ' not in temp[i][0] and '膏 ' not in temp[i][0] and '丸 ' not in temp[i][0] and '散 ' not in temp[i][0] and '饮 ' not in temp[i][0] and '丹 ' not in temp[i][0] ) :
                        index=temp[i][0].index(' ')
                        new1=temp[i][0][:index]
                        new1=new1.replace('(span)','')
                        new2=temp[i][0][index+1:]
                        temp[i-1][-1]=temp[i-1][-1]+'。'+new1
                        temp[i-1].append(new2)
                        temp[i]=[]

In [ ]:
#爬取的数据存入json
filename='new测试4.26.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(new_dict2,file_obj,ensure_ascii=False,indent = 4)

In [38]:
#读取json中爬取的数据
with open('new测试4.26.json',encoding="utf-8") as file_obj:
    new_dict2=json.load(file_obj)

## 4.27

In [39]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if len(temp[i])==2 and '（'==temp[i][1][0] and '）'==temp[i][1][-1] and temp[i][1].count('（')==1 and ' ' in temp[i][0] and ' 治' in temp[i][0]:
                        index=temp[i][0].index(' ')
                        new1=temp[i][0][:index]
                        new2=temp[i][0][index+1:]
                        temp[i][0]=new1
                        temp[i].append(new2)

In [40]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if len(temp[i])==2 and '（'==temp[i][1][0] and '）'==temp[i][1][-1] and temp[i][1].count('（')==1 and ' ' in temp[i][0]:
                        if '（方见' not in temp[i][1]:
                            temp[i].extend(temp[i+1])
                            temp[i+1]=[]

In [41]:
#爬取的数据存入json
filename='new测试4.27.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(new_dict2,file_obj,ensure_ascii=False,indent = 4)

In [57]:
#读取json中爬取的数据
with open('new测试4.27.json',encoding="utf-8") as file_obj:
    new_dict2=json.load(file_obj)

In [59]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if len(temp[i])==2 and temp[i][1].count('（')==1 and '（'==temp[i][1][0] and '）'==temp[i][1][-1] and '方见' not in temp[i][1]:
                        temp[i].extend(temp[i+1])
                        temp[i+1]=[]

In [61]:
#爬取的数据存入json
filename='new测试4.27_2.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(new_dict2,file_obj,ensure_ascii=False,indent = 4)

## 4.28

In [30]:
#读取json中爬取的数据
with open('new测试4.27_2.json',encoding="utf-8") as file_obj:
    new_dict2=json.load(file_obj)

In [ ]:
检查每个方最后一个是否有缺的 把治 疗全分走

In [31]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    for j in range(0,len(temp[i])):
                        temp[i][j]=temp[i][j].replace('(span)','')

In [32]:
spe={}
n_spe={}
for key in new_dict2.keys():
    spe[key]={}
    n_spe[key]={}
    for name in new_dict2[key].keys():
        spe[key][name]={}
        n_spe[key][name]={}
        for cate in new_dict2[key][name].keys():
            spe[key][name][cate]={}
            n_spe[key][name][cate]={}
            if '论' in new_dict2[key][name][cate]:
                spe[key][name][cate]['论']=new_dict2[key][name][cate]['论']
                del new_dict2[key][name][cate]['论']
            if '方' in new_dict2[key][name][cate]:
                spe[key][name][cate]['方']=[]
                n_spe[key][name][cate]['方']=[]
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if '治'==temp[i][0][0] or '疗'==temp[i][0][0] or '主'==temp[i][0][0]:
                        spe[key][name][cate]['方'].append(temp[i])
                        new_dict2[key][name][cate]['方'][i]=[]
                        continue
                        
                    if '丸'==temp[i][0][-1] or '散'==temp[i][0][-1] or '汤'==temp[i][0][-1] or '饮'==temp[i][0][-1] or '酒'==temp[i][0][-1] or '膏'==temp[i][0][-1] or '煎'==temp[i][0][-1] or '丹'==temp[i][0][-1] or '子'==temp[i][0][-1]:
                        n_spe[key][name][cate]['方'].append(temp[i])
                        new_dict2[key][name][cate]['方'][i]=[]
                        continue

In [33]:
#爬取的数据存入json
filename='临时_方4.28.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(new_dict2,file_obj,ensure_ascii=False,indent = 4)

In [34]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if len(temp[i])!=0:
                        if '丸治' in temp[i][0] or '散治' in temp[i][0] or '汤治' in temp[i][0] or '饮治' in temp[i][0] or '酒治' in temp[i][0] or '膏治' in temp[i][0] or '煎治' in temp[i][0] or '丹治' in temp[i][0] or '子治' in temp[i][0]:
                            index=temp[i][0].index('治')
                            new1=temp[i][0][:index]
                            new2=temp[i][0][index:]
                            new=[]
                            new.append(new1)
                            new.append(new2)
                            new.extend(temp[i][1:])
                            temp[i]=new

In [35]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if len(temp[i])!=0:
                        if '丸疗' in temp[i][0] or '散疗' in temp[i][0] or '汤疗' in temp[i][0] or '饮疗' in temp[i][0] or '酒疗' in temp[i][0] or '膏疗' in temp[i][0] or '煎疗' in temp[i][0] or '丹疗' in temp[i][0] or '子疗' in temp[i][0]:
                            index=temp[i][0].index('疗')
                            new1=temp[i][0][:index]
                            new2=temp[i][0][index:]
                            new=[]
                            new.append(new1)
                            new.append(new2)
                            new.extend(temp[i][1:])
                            temp[i]=new

In [36]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if len(temp[i])!=0:
                        if '治'==temp[i][0][0] or '疗'==temp[i][0][0] or '主'==temp[i][0][0]:
                            spe[key][name][cate]['方'].append(temp[i])
                            new_dict2[key][name][cate]['方'][i]=[]
                            continue
                        
                        if '丸'==temp[i][0][-1] or '散'==temp[i][0][-1] or '汤'==temp[i][0][-1] or '饮'==temp[i][0][-1] or '酒'==temp[i][0][-1] or '膏'==temp[i][0][-1] or '煎'==temp[i][0][-1] or '丹'==temp[i][0][-1] or '子'==temp[i][0][-1] or '草'==temp[i][0][-1] or '粥'==temp[i][0][-1] or '汁'==temp[i][0][-1] or '贴'==temp[i][0][-1]:
                            n_spe[key][name][cate]['方'].append(temp[i])
                            new_dict2[key][name][cate]['方'][i]=[]
                            continue

In [37]:
#爬取的数据存入json
filename='临时_方4.28.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(new_dict2,file_obj,ensure_ascii=False,indent = 4)

In [38]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if len(temp[i])!=0:
                        if ' 疗' in temp[i][0]:
                            index=temp[i][0].index(' 疗')
                            new1=temp[i][0][:index]
                            new2=temp[i][0][index+1:]
                            new=[]
                            new.append(new1)
                            new.append(new2)
                            new.extend(temp[i][1:])
                            temp[i]=new

In [39]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if len(temp[i])!=0:
                        if ' 治' in temp[i][0]:
                            index=temp[i][0].index(' 治')
                            new1=temp[i][0][:index]
                            new2=temp[i][0][index+1:]
                            new=[]
                            new.append(new1)
                            new.append(new2)
                            new.extend(temp[i][1:])
                            temp[i]=new

In [40]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if len(temp[i])!=0:
                        if '治'==temp[i][0][0] or '疗'==temp[i][0][0] or '主'==temp[i][0][0]:
                            spe[key][name][cate]['方'].append(temp[i])
                            new_dict2[key][name][cate]['方'][i]=[]
                            continue
                        
                        if '丸'==temp[i][0][-1] or '散'==temp[i][0][-1] or '汤'==temp[i][0][-1] or '饮'==temp[i][0][-1] or '酒'==temp[i][0][-1] or '膏'==temp[i][0][-1] or '煎'==temp[i][0][-1] or '丹'==temp[i][0][-1] or '子'==temp[i][0][-1] or '草'==temp[i][0][-1] or '粥'==temp[i][0][-1] or '汁'==temp[i][0][-1] or '贴'==temp[i][0][-1]:
                            n_spe[key][name][cate]['方'].append(temp[i])
                            new_dict2[key][name][cate]['方'][i]=[]
                            continue

In [41]:
#爬取的数据存入json
filename='临时_方4.28.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(new_dict2,file_obj,ensure_ascii=False,indent = 4)

In [42]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if len(temp[i])!=0:
                        if '（' in temp[i][0]:
                            if '（出' in temp[i][0] or '（一' in temp[i][0] or '（又' in temp[i][0] or '（即' in temp[i][0] or '（医' in temp[i][0]:
                                index=temp[i][0].index('（')
                                new1=temp[i][0][:index]
                                new2=temp[i][0][index:]
                                new=[]
                                new.append(new1)
                                new.append(new2)
                                new.extend(temp[i][1:])
                                temp[i]=new

In [43]:
lst=['丸 ' ,'散 ','汤 ','饮 ','酒 ','膏 ','煎 ','丹 ','子 ']

In [44]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if len(temp[i])!=0:
                        for st in lst:
                            if st in temp[i][0]:
                                index=temp[i][0].index(st)
                                new1=temp[i][0][:index+1]
                                new2=temp[i][0][index+2:]
                                new=[]
                                new.append(new1)
                                new.append(new2)
                                new.extend(temp[i][1:])
                                temp[i]=new                            

In [45]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if len(temp[i])!=0:
                        if '治'==temp[i][0][0] or '疗'==temp[i][0][0] or '主'==temp[i][0][0]:
                            spe[key][name][cate]['方'].append(temp[i])
                            new_dict2[key][name][cate]['方'][i]=[]
                            continue
                        
                        if '丸'==temp[i][0][-1] or '散'==temp[i][0][-1] or '汤'==temp[i][0][-1] or '饮'==temp[i][0][-1] or '酒'==temp[i][0][-1] or '膏'==temp[i][0][-1] or '煎'==temp[i][0][-1] or '丹'==temp[i][0][-1] or '子'==temp[i][0][-1] or '草'==temp[i][0][-1]:
                            n_spe[key][name][cate]['方'].append(temp[i])
                            new_dict2[key][name][cate]['方'][i]=[]
                            continue

In [75]:
'''for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if len(temp[i])!=0:
                        if ' ' in temp[i][0]:
                            index=temp[i][0].index(' ')
                            new1=temp[i][0][:index]
                            new2=temp[i][0][index+1:]
                            last=temp[i][1:]
                            temp[i]=[]
                            temp[i].append(new1)
                            temp[i].append(new2)
                            temp[i].extend(last)'''

In [46]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if len(temp[i])!=0:
                        if '治'==temp[i][0][0] or '疗'==temp[i][0][0] or '主'==temp[i][0][0]:
                            spe[key][name][cate]['方'].append(temp[i])
                            new_dict2[key][name][cate]['方'][i]=[]
                            continue
                        
                        if '丸'==temp[i][0][-1] or '散'==temp[i][0][-1] or '汤'==temp[i][0][-1] or '饮'==temp[i][0][-1] or '酒'==temp[i][0][-1] or '膏'==temp[i][0][-1] or '煎'==temp[i][0][-1] or '丹'==temp[i][0][-1] or '子'==temp[i][0][-1] or '草'==temp[i][0][-1] or '粥'==temp[i][0][-1] or '汁'==temp[i][0][-1] or '贴'==temp[i][0][-1] or '饼'==temp[i][0][-1]:
                            n_spe[key][name][cate]['方'].append(temp[i])
                            new_dict2[key][name][cate]['方'][i]=[]
                            continue

In [47]:
#爬取的数据存入json
filename='临时_方4.28.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(new_dict2,file_obj,ensure_ascii=False,indent = 4)

In [48]:
#爬取的数据存入json
filename='不特殊4.28.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(n_spe,file_obj,ensure_ascii=False,indent = 4)

In [17]:
def clean():
    for key in new_dict2.keys():
        for name in new_dict2[key].keys():
            for cate in new_dict2[key][name].keys():
                if '方' in new_dict2[key][name][cate].keys():
                    temp=new_dict2[key][name][cate]['方']
                    new=[]
                    for i in temp:
                        if len(i)!=0:
                            new.append(i)
                    new_dict2[key][name][cate]['方']=new
                    
    for key in new_dict2.keys():
        for name in new_dict2[key].keys():
            for cate in new_dict2[key][name].keys():
                if '方' in new_dict2[key][name][cate].keys():
                    if len(new_dict2[key][name][cate]['方'])==0:
                        del new_dict2[key][name][cate]['方']
                        
                        
    for key in new_dict2.keys():
        for name in new_dict2[key].keys():
            new={}
            for cate in new_dict2[key][name].keys():
                if len(new_dict2[key][name][cate])!=0:
                    new[cate]=new_dict2[key][name][cate]
            new_dict2[key][name]=new

In [49]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                new=[]
                for i in temp:
                    if len(i)!=0:
                        new.append(i)
                new_dict2[key][name][cate]['方']=new
                
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                if len(new_dict2[key][name][cate]['方'])==0:
                    del new_dict2[key][name][cate]['方']
                    

for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        new={}
        for cate in new_dict2[key][name].keys():
            if len(new_dict2[key][name][cate])!=0:
                new[cate]=new_dict2[key][name][cate]
        new_dict2[key][name]=new

In [52]:
#爬取的数据存入json
filename='临时_方4.28.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(new_dict2,file_obj,ensure_ascii=False,indent = 4)

In [53]:
#爬取的数据存入json
filename='特殊4.28.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(spe,file_obj,ensure_ascii=False,indent = 4)

## 4.29 以方结尾 通过判断几行来判断是否要入 n_spe

In [10]:
#读取json中爬取的数据
with open('临时_方4.28.json',encoding="utf-8") as file_obj:
    new_dict2=json.load(file_obj)

In [11]:
#读取json中爬取的数据
with open('不特殊4.28.json',encoding="utf-8") as file_obj:
    n_spe=json.load(file_obj)

In [12]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if len(temp[i])!=0:
                        if '治'==temp[i][0][0] or '疗'==temp[i][0][0] or '主'==temp[i][0][0]:
                            spe[key][name][cate]['方'].append(temp[i])
                            new_dict2[key][name][cate]['方'][i]=[]
                            continue
                        
                        if '丸'==temp[i][0][-1] or '散'==temp[i][0][-1] or '汤'==temp[i][0][-1] or '饮'==temp[i][0][-1] or '酒'==temp[i][0][-1] or '膏'==temp[i][0][-1] or '煎'==temp[i][0][-1] or '丹'==temp[i][0][-1] or '子'==temp[i][0][-1] or '草'==temp[i][0][-1] or '粥'==temp[i][0][-1] or '汁'==temp[i][0][-1] or '贴'==temp[i][0][-1] or '饼'==temp[i][0][-1]:
                            n_spe[key][name][cate]['方'].append(temp[i])
                            new_dict2[key][name][cate]['方'][i]=[]
                            continue

In [13]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if len(temp[i])>=3 and '方'==temp[i][0][-1]:
                        n_spe[key][name][cate]['方'].append(temp[i])
                        new_dict2[key][name][cate]['方'][i]=[]

In [14]:
#爬取的数据存入json
filename='临时_方4.28.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(new_dict2,file_obj,ensure_ascii=False,indent = 4)

In [19]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if len(temp[i])>=3 and '方。'in temp[i][0] and '治' in temp[i][1]:
                        n_spe[key][name][cate]['方'].append(temp[i])
                        new_dict2[key][name][cate]['方'][i]=[]

In [20]:
#爬取的数据存入json
filename='临时_方4.29.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(new_dict2,file_obj,ensure_ascii=False,indent = 4)

In [21]:
#爬取的数据存入json
filename='不特殊4.29.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(n_spe,file_obj,ensure_ascii=False,indent = 4)

In [54]:
#读取json中爬取的数据
with open('临时_方4.29.json',encoding="utf-8") as file_obj:
    new_dict2=json.load(file_obj)

In [55]:
#读取json中爬取的数据
with open('不特殊4.29.json',encoding="utf-8") as file_obj:
    n_spe=json.load(file_obj)

In [56]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if len(temp[i])!=0:
                        if '治'==temp[i][0][0] or '疗'==temp[i][0][0] or '主'==temp[i][0][0] or '法治'==temp[i][0][0]:
                            spe[key][name][cate]['方'].append(temp[i])
                            new_dict2[key][name][cate]['方'][i]=[]
                            continue
                        
                        if '丸'==temp[i][0][-1] or '散'==temp[i][0][-1] or '汤'==temp[i][0][-1] or '饮'==temp[i][0][-1] or '酒'==temp[i][0][-1] or '膏'==temp[i][0][-1] or '煎'==temp[i][0][-1] or '丹'==temp[i][0][-1] or '子'==temp[i][0][-1] or '草'==temp[i][0][-1] or '粥'==temp[i][0][-1] or '汁'==temp[i][0][-1] or '贴'==temp[i][0][-1] or '饼'==temp[i][0][-1]:
                            n_spe[key][name][cate]['方'].append(temp[i])
                            new_dict2[key][name][cate]['方'][i]=[]
                            continue

In [57]:
#爬取的数据存入json
filename='临时_方4.29_2.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(new_dict2,file_obj,ensure_ascii=False,indent = 4)
    
#爬取的数据存入json
filename='不特殊4.29_2.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(n_spe,file_obj,ensure_ascii=False,indent = 4)

In [59]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if len(temp[i])>=3 and '治' in temp[i][1]:
                        n_spe[key][name][cate]['方'].append(temp[i])
                        new_dict2[key][name][cate]['方'][i]=[]

In [61]:
clean()

In [62]:
#爬取的数据存入json
filename='临时_方4.29_2.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(new_dict2,file_obj,ensure_ascii=False,indent = 4)

In [71]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if '。'==temp[i][0][-1]:
                        temp[i][0]=temp[i][0][:-1]

In [23]:
def change():
    for key in new_dict2.keys():
        for name in new_dict2[key].keys():
            for cate in new_dict2[key][name].keys():
                if '方' in new_dict2[key][name][cate].keys():
                    temp=new_dict2[key][name][cate]['方']
                    for i in range(0,len(temp)):
                        if len(temp[i])!=0:
                            if '治'==temp[i][0][0] or '疗'==temp[i][0][0] or '主'==temp[i][0][0] or '法治'==temp[i][0][0] or '止'==temp[i][0][0] or '通'==temp[i][0][0]:
                                spe[key][name][cate]['方'].append(temp[i])
                                new_dict2[key][name][cate]['方'][i]=[]
                                continue
                        
                            if '丸'==temp[i][0][-1] or '散'==temp[i][0][-1] or '汤'==temp[i][0][-1] or '饮'==temp[i][0][-1] or '酒'==temp[i][0][-1] or '膏'==temp[i][0][-1] or '煎'==temp[i][0][-1] or '丹'==temp[i][0][-1] or '子'==temp[i][0][-1] or '草'==temp[i][0][-1] or '粥'==temp[i][0][-1] or '汁'==temp[i][0][-1] or '贴'==temp[i][0][-1] or '饼'==temp[i][0][-1] or '根'==temp[i][0][-1] or '粉'==temp[i][0][-1]:
                                n_spe[key][name][cate]['方'].append(temp[i])
                                new_dict2[key][name][cate]['方'][i]=[]
                                continue

In [73]:
change()

In [74]:
clean()

In [75]:
#爬取的数据存入json
filename='临时_方4.29_2.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(new_dict2,file_obj,ensure_ascii=False,indent = 4)
    
#爬取的数据存入json
filename='不特殊4.29_2.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(n_spe,file_obj,ensure_ascii=False,indent = 4)
    
#爬取的数据存入json
filename='特殊4.29.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(spe,file_obj,ensure_ascii=False,indent = 4)

In [83]:
#读取json中爬取的数据
with open('临时_方4.29_2.json',encoding="utf-8") as file_obj:
    new_dict2=json.load(file_obj)

In [85]:
change()
clean()

In [86]:
#爬取的数据存入json
filename='临时_方4.29_2.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(new_dict2,file_obj,ensure_ascii=False,indent = 4)
    
#爬取的数据存入json
filename='不特殊4.29_2.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(n_spe,file_obj,ensure_ascii=False,indent = 4)
    
#爬取的数据存入json
filename='特殊4.29.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(spe,file_obj,ensure_ascii=False,indent = 4)

# 4.29

In [26]:
#读取json中爬取的数据
with open('临时_方4.30.json',encoding="utf-8") as file_obj:
    new_dict2=json.load(file_obj)
    
#读取json中爬取的数据
with open('不特殊4.30.json',encoding="utf-8") as file_obj:
    n_spe=json.load(file_obj)
    
#读取json中爬取的数据
with open('特殊4.29.json',encoding="utf-8") as file_obj:
    spe=json.load(file_obj)

In [27]:
change()
clean()

In [28]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if '方治' in temp[i][0]:
                        index=temp[i][0].index('方治')
                        new1=temp[i][0][:index+1]
                        new2=temp[i][0][index+2:]
                        new=[]
                        new.append(new1)
                        new.append(new2)
                        new.extend(temp[i][1:])
                        n_spe[key][name][cate]['方'].append(new)
                        new_dict2[key][name][cate]['方'][i]=[]

In [29]:
change()
clean()

In [30]:
#爬取的数据存入json
filename='临时_方4.30_2.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(new_dict2,file_obj,ensure_ascii=False,indent = 4)
    
#爬取的数据存入json
filename='不特殊4.30_2.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(n_spe,file_obj,ensure_ascii=False,indent = 4)
    
#爬取的数据存入json
filename='特殊4.30_2.json'
with open(filename,'w',encoding='utf-8') as file_obj:
    json.dump(spe,file_obj,ensure_ascii=False,indent = 4)

In [2]:
#读取json中爬取的数据
with open('临时_方4.30_2.json',encoding="utf-8") as file_obj:
    new_dict2=json.load(file_obj)
    
#读取json中爬取的数据
with open('不特殊4.30_2.json',encoding="utf-8") as file_obj:
    n_spe=json.load(file_obj)
    
#读取json中爬取的数据
with open('特殊4.30_2.json',encoding="utf-8") as file_obj:
    spe=json.load(file_obj)

In [3]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if '丸' in temp[i][0] or '散' in temp[i][0] or '汤' in temp[i][0] or '饮' in temp[i][0] or '酒' in temp[i][0] or '膏' in temp[i][0] or '煎' in temp[i][0] or '丹'in temp[i][0] or '子' in temp[i][0] or '草' in temp[i][0] or '粥' in temp[i][0] or '汁' in temp[i][0] or '贴' in temp[i][0] or '饼' in temp[i][0] or '根' in temp[i][0] or '粉' in temp[i][0]:
                        print(temp[i])
                        print(' ')

['消酒进食方', '生姜（二两） 白术（五钱） 草果（一两） 缩砂〔二（一）两〕 檀香（五钱） 炒 盐（五钱） 上']
 
['熟水草果饮法', '乌梅肉（四两） 草果 干姜（炮各三两） 赤茯苓〔一（二）两〕 甘草（炙半两） (br/)上 咀。 每用半两。水一盏半。煎至一盏去滓。瓷器盛如熟水温和。随意服之。']
 
['肠满。四肢倦。行立艰。食以呕。多冒。减饮食。或渴者是也。宜用后方', '（出 华佗中 大黄（一两湿纸十重包裹煨令香熟切作片子） 槟榔（一枚） 木香（一分）(br/)上为末 生蜜丸 桐子 桂（半两不焙） 槟榔（一个） 黑牵牛（四两生取末二两节不用）(br/)上为末。蜜酒调 二钱。 以利']
 
['安脾脏进饮食益气汤方', '（出杨子建万全护命方） 黄橘皮 人参（各半两） 丁香 肉豆蔻 甘草 枇杷叶（拭去毛） 藿香（各一分）(br/)上 细 杵罗为末。每服二钱。水一盏。煎取八分。空心服。']
 
['补肾腰子法', '（出鸡峰方）治老人肾脏虚寒。即其内肾以寒虚自结实硬。虽服补药。并不入用 右羊腰子一对。水半碗。用杜仲阔一寸长二寸许一片。同煮腰子软。空心切食。令人内 肾柔']
 
['膈气变翻胃。用剪红丸。吐出淤血及下虫而愈。真方并附', '（永类钤方）', '雄黄（五钱） 木香（五钱） 槟榔 三棱 莪术（煨） 陈皮 贯众（去毛各一两） 大黄（春 三两', '又方', '芫花（一两醋煮） 巴豆（二十一粒去油） 甘遂（一两）(br/)上为末。糊丸如梧桐子大。 候干 用红 醋炙']
 
['澡豆悦面急面皮。去 粉刺方', '蔓荆子 白芷 芎? 皂角末 葳蕤 白术（各五两） 冬瓜仁 栀子仁 栝蒌仁 荜 豆（三升 别 捣如泥浆水。']
 
['炼草留颜方', '用五月五日收取益母草。俗呼为之郁嗅草。曝干烧灰。取草时勿令根上有土。即无效。 烧之 团 丸 之 研 童']
 
['面膏方 令人面色悦泽。桃花红光', '防风 芎? 白芷 白僵蚕 蜀水花 白蔹 细辛 茯苓 本 葳蕤 青木香 辛夷 仁当归 土瓜根 栝蒌（各三分） 桃仁（六分） 猪脂 鹅脂 羊肾（各一升）(br/)上细 切裹。 酒二升。浸一日一夜。便纳脂急火煎之。三上三下。然后慢火一夜。药成去滓。以水石粉三分。纳脂中。以柳木篦熟搅。任用之。']
 
['磁石酒浸方', '治风邪入于脑。或入于耳。久而不散。经

In [33]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if len(temp[i])==2:
                        if '揩' in temp[i][0] or '主治' in temp[i][0]:
                            print(temp[i])
                            print(' ')

['黑髭揩齿方', '（出圣惠方） 鲤鱼（一头可盛得药者去肠肚不洗） 生地黄（八两竹刀细切） 诃黎勒皮（二两） 茜 草根（一 纳入鱼 腹中 研。 内']
 
['乌髭鬓揩牙法', '（出圣济总录） 升麻（一两） 诃黎勒（二枚去核为末） 白盐花（半分） 粟 饭（一合） 麻糁末（四合取 第一 拌匀 每 湿 肠 若']
 
['乌髭揩牙药。能反白为黑', '（出百一选方） 蓖麻子（四十九个） 生干地黄（一两） 龙骨（半两） 细辛（一分） 青盐（半两） 上用大 栝 烟']
 
['乌髭鬓常用揩牙药', '（出百一选方） 青盐（一两别研） 杏仁〔二（三）两去皮尖〕 熟干地黄（洗净） 旌德乌头（各一两）(br/)上为 有小 每']
 
['揩牙乌髭方', '（出圣济总录） 皂荚（怀州者十条） 生姜（自然汁） 地黄（自然汁） 各半盏。(br/)上以二汁和匀。旋 刷皂荚 于火']
 
['乌髭鬓揩牙药', '（出百一选方） 用不蛀皂角。切作小段。长寸许。熔黑铅令销成水。投皂角炒如炭。不可太过。恐成灰。 取']
 
['乌髭揩牙药', '（出卫生家宝方） 苦参（半两） 青黛（一两如螺色青者） 青盐（一两）(br/)上先以苦参为末。次研青黛。 又研 青盐 州。陛陆辞日。神宗怪其髭鬓尽黑。对云用此方。遂录进呈。']
 
['紫金散揩牙乌髭鬓。兼治牙齿一切疾病', '麻糁 地黄 青盐 皂荚 东引桃枝 柳枝 桑枝 马齿 上件各等分。取盛三升瓦罐 子一 纸箸 升麻 牙']
 
['华山石刻散揩齿乌髭鬓。及治牙齿诸疾。若用此药。永不患病', '鸡肠草 旱莲草 茜草根 晚蚕砂 白矾 青盐 不 皂荚 诃子（各等分）(br/)上 咀。 如麻 末。']
 
['七宝散揩齿令白净', '海蛤 琥珀 珍珠 白石英 玛瑙 光明（各一两） 麝香（一分）(br/)上为细散。于乳 钵内重研']
 
['升麻散揩牙如常时', '升麻 生地黄 皂荚 干石榴子 柳枝 巨胜（各半两）(br/)上锉细。入瓷油瓶中。以盐 泥固济']
 
['金牙散揩齿', '金牙 蟾酥 细辛 黄芩 白芷（各半两） 升麻（一两）(br/)上为末。别用 荞麦 再为细末。别研龙脑、麝香、各少许。具砂、鹤黄、各半升。再同研匀。用瓷盒盛。逐日点 药揩齿']
 
['莲子草散揩齿', '莲子草 升麻 牛膝 茜草 丁香（各半两） 生地黄 上入瓶子内。泥封头

In [21]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if len(temp[i])==2:
                        if '治' not in temp[i][1]:
                            print(temp[i])
                            print(' ')

['方', '秦艽（去苗土） 干姜（炮） 桔梗（炒） 附子（炮裂去皮脐各一两） 当归（切焙） 天门冬 （去 脐尖） 细 风 （去叉） 五味子（各一钱） (br/)上捣。罗为散。每服一钱。温酒调下。日三。不拘时候。']
 
['搓滚舒筋法', '用大筒长一尺许。两头通一杖子。穿过。以绳系两头于坐椅上。每坐。则以脚踏竹筒上。 以']
 
['润心肺', '（出本草） 以丹砂服之。']
 
['宁心志清神', '（出本草） 以石莲子取其肉。于砂盆中干擦之。去浮上赤色。留青心为末。入龙脑为汤。点服。']
 
['解烦闷', '（本草）以白粱粟。经三五年陈者。或呼为粢米。作粉食之。']
 
['除胸膈热痰', '用三白草。捣。绞汁服。令人吐逆。']
 
['补心虚治健忘，令耳目聪明。宜服此方', '（出圣惠方） 菖蒲 磁石（烧醋淬七遍捣碎细研如粉） 茯神 熟干地黄 薯蓣（各二两） 人参（去 芦头） 杵。丸如梧桐']
 
['养命开心益智方', '（出千金方） 干地黄 人参 茯苓（各二两） 苁蓉 菟丝子 远志（各三两） 蛇床子〔二两（分）〕 上 药']
 
['养生方', '男子勿北首卧。神魂不安。多愁忘。']
 
['镇心', '（出本草） 用蛇黄如入药。烧赤三四次。醋淬。飞研用之。']
 
['青牛道士封君达。传鲍陂山人。治心常忪悸。行惧往。忘前失后不定', '（余居 士选奇方） 白檀（十二分） 甘草（十分） 石菖蒲 犀角 天竺黄 干熟地黄 苏合香（各四分） 桂心 食后 含化']
 
['脾痛气痛方', '（直指方） 良姜 陈皮（各半两） 丁香 石菖蒲 甘草（炙各一钱）(br/)上锉散。每服三钱。水一大盏。煎']
 
['脾痛单方', '（出朱氏集验方） 张忠顺。盛夏调官都城。苦热。食冰雪过多。又饮木瓜浆。积冷于中。遂感脾痛之疾。 药不 一粒 脱']
 
['又十便 治腹胀膨，鼓之有声。喘息不便。由上下不升降。痞无归', '三焦浑乱用。 浓朴（四两姜制） 枳实〔一（三）两炒〕 桂心（一两）(br/)上粗散。每服五钱。水二盏。煎至一']
 
['消酒进食方', '生姜（二两） 白术（五钱） 草果（一两） 缩砂〔二（一）两〕 檀香（五钱） 炒 盐（五钱） 上']
 
['调和胃气', '（本草） 以干姜去核。于铛锅中。微火缓逼干为末。量多少。入生姜末为汤点。服初六日得处暑 节。

['补腰脚令健', '（出本草） 取女贞皮。以浸酒服。']
 
['补虚强脚益肾', '（出本草） 以零余子食之。']
 
['老', '松脂（三十两法煮白） 茯苓 甘菊花 柏子仁（各十两）(br/)上为末。炼蜜和丸。如梧桐子大。']
 
['补虚益髓', '用鹿麋角可五寸。截之中破。炙令黄香为末。和酒。空心服三钱匕。若卒心痛。一服立 瘥。 茸甚']
 
['阳炼法', '（出危氏方） 用小便不计多少。大约两桶一担。先以清水煮好皂荚浓汁。以布绞去滓。每小便一担桶。 入 清 数 下 清汁。 火。 围养七 服三十 不效。 小锅便']
 
['阴炼法', '（出圣济总录） 用小便三五石。夏月虽腐败亦堪用。分置大盆中。以新水一半以上相和。旋转搅数百匝。 放 石 之 如梧桐子大。曝干。每服三十丸。温酒下。']
 
['补虚劳。益髓长肌。悦颜色。令人肥健', '用鹿角胶炙捣为末。以酒服方寸匕。早午晚日三服。增至二三匕效。']
 
['强筋健髓', '（出本草） 以苁蓉鳝鱼为末。黄精酒丸服之。力可十倍。']
 
['补益精气。强盛阴道', '（出本草） 以萝卜生啖。亦蒸煮食之。']
 
['补益精气。强盛阴道', '（出本草） 以枸杞叶作羹食之。']
 
['益筋力。去伏热', '（出本草） 以芹菜作齑菹。及煮食并得。']
 
['容颜不老丹方', '一斤生姜半斤枣。二两白盐三两草。丁香沉香各半两。四两茴香一处捣。煎也好。点也 好。']
 
['炼松脂法', '（出圣济总录） 用松脂十斤。以桑柴灰汁。煮三五次。候色白止。即用布滤入冷水中。其精者入水即凝。 其 用']
 
['补虚损。益颜色', '用薯蓣于砂盆中细研。然后下于铫中。先以酥一大匙。熬令香。次旋添酒一盏。煎搅令 匀。']
 
['除百病。好容色', '（一云益颜色） 用桃花以酒渍饮之。三月三采。又桃仁服之长生。']
 
['补益方', '用天门冬细切。阴干捣下筛。酒调三钱。每日五六进之。二百日知。可以强筋髓。驻颜 色']
 
['除风补衰。久服不老。令人好颜色。肥健益阳道。温中去痼疾', '（出本草） 用菊花水饮之。荆州记云。郦县菊水太尉胡广。久患风羸。常汲饮此水。后疾遂瘳。']
 
['令人好颜色', '（出本草） 以百花上露饮之。']
 
['令人肥白。美颜色', '以麋骨煮作汁。酿酒饮之。']
 
['补虚损。益

['服石法', '粗白石英一大片敲碎。颗粒如酸枣核大。不用全。取白石颗先于砂盆中和粗磊磊砂。使 壮儿 即 熟 取 家 恐 此至日午左侧。即便转出。为新石推陈石下讫。还依大家食时即餐饭。若自知病羸。至夜。 食前又服七颗。依前法吞一夜。令在小腹下温齐脚。明日平明。先便转陈石。总与石下讫。 又朝法夜法服之。此石常在小腹内。仍附仓门。但小腹温热。于四肢膀胱头目脑髓肤体之内。元无石气。欲从何而作。丈夫妇人多有积冷。若下热。必须上冷。若上下俱冷。胃口不下 食。便成消渴。至若上下俱热。头面生疮。唇干眼赤。手脚枯槁。皮毛浮起。不久成骨蒸。 凡人必须上下焦冷热气息调和。筋脉通达。若上热下冷。必有痼积。服石之后。即下热。自 然上冷。骨气坚实。腰肾强健。万病自除。石气力得。三年以来。若不得力。十斤亦须常吃。若得力。吃一斤即止也。']
 
['白石英粉', '以白石英五两。微捣研。以 醋五大升。于不津瓷器中。盖头埋屋北阴处。经二十日出。 泻 还 安不津器中。重汤煮令乳竭止。依法方用和后丸。']
 
['地黄酒法', '用生地黄。切五小升。乌豆三小升。以无灰清酒。渍经五宿。取服前丸药。']
 
['羊肉蒸石英服饵法', '精羊肉（一斤） 白石英（三两）(br/)上先取肉劈作两段。钻作孔。纳石英肉中。还相和。 用荷 姜等 春夏']
 
['石英汁作姜豉服饵法', '白石英（二两） 肥猪肉（三斤） (br/)上以水八升。煮石英。取五升。量煮猪肉烂熟为度。 取 肉汁 第二 八']
 
['石英磁石浸酒饵法', '白石英（五两泽州好者） 磁石（五两去毛石连碱多者十两亦得） (br/)上各捣令碎。各用 两 重。白 微有酒 并随所 酒三五日后。即渐添一二升。常令瓶满。所加药。疑力尽者。任换之。经三五个月。疑石力 稍微。']
 
['五乖', '（出圣惠方） 重衣更寒。一乖。（凡人寒衣即暖服石人宜薄衣若重衣即寒经云热极生寒故云一乖） 饥则生臭。二乖。（平人饮食不消作生食气服石人忍饥失食即有生食气与常人殊故云二 乖）极则自劳。三乖。（平人有所疲极即须消息恬养服石人夕坐久坐疲极须自劳适散石气即 畅 故 温即泄利。四乖。（平人因冷乃利得暖便愈服石人温则泄冷则瘥故云四乖） 饮食欲寒。五乖。（平人食温暖则五内调和服石人食饮欲寒乃得安稳故云五乖千金翼方 一名']
 
['七急', '（出圣惠方） 当洗勿失时。一急。（

['枯瘤方', '用铜绿为细末。刺破瘤。糁在上。以膏药贴之。']
 
['赘瘤方', '用绵纹大黄。醋磨。涂瘤上。渐消。']
 
['白瘤方', '令先极搔刮。以绳缚之。即愈。又取东向木空中水。刮瘤上洗之。凡一日三次。即愈。']
 
['做土法', '砒黄（研为末三两） 木鳖子（半两去壳） 巴豆（半两不去油） 硼砂（另为末二两） (br/)上砒黄一 于磁器盛']
 
['枯瘤子药方', '桑柴灰（二碗） 腻白锻石（三碗） 朽木（三两） 干桑耳（三两） 草乌头（半斤）川乌头（四 敷之。']
 
['洗方', '（出危氏方） 黄连 黄芩 荆芥 蛇床子（各一两） 侧柏叶（四两） 槐条 镜面草 蚵 草（各一握）']
 
['洗痔方', '海桐皮 荆芥 甘草节 苦参 葱 黑豆(br/)上等分。锉成粗粒。水五升。煎汤洗。凡涂 药一']
 
['痔疮方', '蛇蜕 萆麻子 荆芥 油头发 皂角 穿山甲（各等分）(br/)上先用轻粉煎汤。洗净疮口 后。用']
 
['熏洗方', '（出经验良方） 槐花（半两） 莲房（一个） 五倍子（三个研细） 荆芥（五钱） 地榆皮（半两） (br/)上为末。每服']
 
['敷痔方', '斑蝥（十个） 轻粉（半钱） 马牙硝（三十枚） 红椒（一钱） 黄皮（半钱） (br/)上为末。先用皂角']
 
['黄柏散涂痔漏', '黄柏 铅丹 黄连 腻粉 白矾(br/)上等分为散。先煎葱汤洗后。用药散一钱匕。涂之。 久']
 
['洗法', '（出三因方） 海桐皮 蛇床子（各一两） 香南藤 葱白（切各三两）(br/)上水一斗。药五两。煎减半。 去滓。 候温。着手轻洗。以绢拭。一法用槐白皮煎。淋渫取佳。']
 
['熏药', '麻花（三钱） 蜈蚣（一条） 水银粉（一钱半） 婆娑石（二钱） 青气矾（二钱） 明矾（二钱） (br/)研细。加血蝎、雄黄']
 
['圣惠方痔药', '白矾 血余 石竹（各半两） 胡椒（二十粒）(br/)上用瓦罐盛。泥固济。猛火 通红。取出。去']
 
['洗痔方', '防风 当归 川芎（各等分） (br/)上锉细。水煎去滓。令熟。温淋洗疮。用软帛拭干。敷 煎药。']
 
['渫药', '马蹄香 苍术 荆芥（各二两） (br/)上锉细。以水三升。煎数沸。先嘘后洗。']
 
['理痔方', '五灵脂（四钱） 腻粉（半两） 麝香（三字）(br/

In [ ]:
['玉粉锭治牙疳蚀损。宣烂臭秽', '锭粉（二钱） 信（一字）(br/)上研匀细。煮白面胳KT 。冷淘了剂为锭子。如绵。阴干为黄末贴疳']
['白牙药治牙齿黄黑。不莹净', '零陵香 香芷 青盐 升麻（各半两） 细辛 麝香 砂锅 石膏（各一两）(br/)上除砂 锅石膏麝']
['紫金散揩牙乌髭鬓。兼治牙齿一切疾病', '麻糁 地黄 青盐 皂荚 东引桃枝 柳枝 桑枝 马齿 上件各等分。取盛三升瓦罐 子一 纸箸 升麻 牙']
['华山石刻散揩齿乌髭鬓。及治牙齿诸疾。若用此药。永不患病', '鸡肠草 旱莲草 茜草根 晚蚕砂 白矾 青盐 不 皂荚 诃子（各等分）(br/)上 咀。 如麻 末。']

['七宝散揩齿令白净', '海蛤 琥珀 珍珠 白石英 玛瑙 光明（各一两） 麝香（一分）(br/)上为细散。于乳 钵内重研']
['升麻散揩牙如常时', '升麻 生地黄 皂荚 干石榴子 柳枝 巨胜（各半两）(br/)上锉细。入瓷油瓶中。以盐 泥固济']
 
['金牙散揩齿', '金牙 蟾酥 细辛 黄芩 白芷（各半两） 升麻（一两）(br/)上为末。别用 荞麦 再为细末。别研龙脑、麝香、各少许。具砂、鹤黄、各半升。再同研匀。用瓷盒盛。逐日点 药揩齿']
 
['莲子草散揩齿', '莲子草 升麻 牛膝 茜草 丁香（各半两） 生地黄 上入瓶子内。泥封头。烧令通 赤。取']
 
['溃牙药 黑牙缝牢牙', '五倍子 酸石榴皮 黄丹 上先用前药末。浆水调贴牙缝上。后药如前。贴二三日见效。']
 
['槐枝散揩齿去风。令白净', '槐枝 巨胜子 青盐 生地黄（各一两） 皂荚（一梃）(br/)上细锉。入一新瓷瓶中盛。 固济。于 用揩齿甚']
 
['刷牙药治诸牙病。频刷着牙。黑缝齿如白玉', '玄蒌 谷精草 松子 白茯苓 上将玄蒌成药。盖之。用麻绳缚定。盐泥固济二指浓。 用炭 用']
['紫金膏专治赤肿风烂眼目', '铜青（研） 砂 硼砂 羚羊角 雄黄 青盐 琥珀 明矾（各七钱半） 当门子 片 脑 胆 斤火 炮 两） 银星石（一两二钱半火炼） 海螵蛸（一两七钱半为末） 轻粉（一钱） 沙糖（三斤） 水（三升）(br/)上以水同蜜。于锅内熬沸去沫。入黄丹。以柳木篦急手搅匀。约熬三两沸。却下炉甘石。 又复搅入乳香、 砂。又下雄黄。次下丁香。再入轻粉。方下片脑。依次第下药。文武火各 熬二三沸。须不住手搅匀成膏。候粘手为度。每用鸡头大。沸汤化开。浸汤半盏。乘热温洗']
['导痰丸专治痰气。服此药免患中风之疾', '黑牵牛（生用头末三钱） 白矾（生用一钱） 猪牙皂角（生用二钱） 半夏（少许） 南 星 （少许）']

In [22]:
for key in new_dict2.keys():
    for name in new_dict2[key].keys():
        for cate in new_dict2[key][name].keys():
            if '方' in new_dict2[key][name][cate].keys():
                temp=new_dict2[key][name][cate]['方']
                for i in range(0,len(temp)):
                    if len(temp[i])==3 and '）'==temp[i][1][-1] and '（'==temp[i][1][0]:
                        print(temp[i])
                        print(' ')

['翻胃始因噎膈所成。有痰母虫瘕。非好手不能奏效', '（出永类钤方）', '附子片（一两） 南星片（一两） 巴豆（二十一粒去皮）(br/)上用糯米一两。同炒变色。去巴 豆。 又 汤 又翻 虎丹。']
 
['牢牙黑髭鬓', '（又名长春绿袍散）', '金丝矾 绿矾 川芎 胆矾（一方用信） 细辛 白茯苓 诃子肉 川百药煎 没石子 酸石了。再']
 
['乌龙髓', '（出百一选方）', '汉防己 绿矾（各一两） 当归（二两） 桑椹子（一百个大黑紫者） (br/)上以罐盛。用麻油十 二两 大妙']
 
['浸油长发。令黑光润泽方', '（出圣惠方）', '生麻油（五升） 干桑椹（一两） 栀子花（一两） 诃黎勒皮（一两） 莲子草（一两） 细辛（半两） 白芷（半两） 本（一两） 零陵香（一两） 白蔹（各一两） 酸石榴皮（一两） 生铁绵裹入油中']
 
['令发速长而黑方', '（出圣惠方）', '乌喙（三两去皮脐生用） 莽草（三两） 续断（三两） 皂荚（二两去黑皮并子） 泽兰（三两） 白术（三两） 细辛（各三两） 辛夷（一两） 柏叶（一两） 防风（一两） 竹叶（一两） 杏仁 猪脂二斤同煎药焦黄。药成去滓。以瓷器盛。每使净洗头了涂之。三十日效。']
 
['令发易长', '（出圣惠方）', '熊脂（一两） 蔓荆子（一两）(br/)上相和令匀。以醋调涂之。发即渐长。']
 
['干洗头药', '（出济生拔粹方）', '甘松 百药煎 五倍子 川芎 薄荷 香白芷 草乌头 藿香 茅香（各等分）(br/)上为末。掺']
 
['蚰蜒入耳方', '（出广南四时摄生方）', '麻油（四两） 蓝根（二两于油内浸五日取出蓝根不用）(br/)上取鸡冠血、乌驴乳。各少许。同入油中。如蚰蜒入左耳。紧塞右耳。向左耳内灌药少许。如经年内有蚰蜒子者。先于耳内灌药少许。后用药。以手从额揩至顶。随即盖覆卧定。其蚰蜒化为黄水流出即瘥。']
 
['肺热口臭口中如胶瓦干。发渴。小便多', '（出永类钤方）', '地骨皮 桑白皮 生黄 山栀子 马兜铃 上甘草煎。食后噙咽。']
 
['取喉中涎', '（出危氏方）', '草乌尖 僵蚕 紫贝 啾 虫 红椒 白矾 山豆根 皂角（各等分）(br/)上为细末。用霜梅拌']
 
['开喉关', '（出危氏方）', '薄荷 山豆根（各五钱） 麝香 脑子（各半钱） 朱砂（三钱）

In [ ]:
粥 汁 贴